In [3]:
import re, textract, pickle, os

In [22]:
# STEP 1: Get file names

# in folder, click on tiny gear icon on top of the window, then select Copy "" pathname
path = '/Volumes/BackupWD2/CbCM data/Luxembourg Archive/2016_unlocked/2016_3/'
#path = '/home/op/kernel/thesis/data/cbm/pdfs/' # Oskar's path
# a package for navigating the operating system (just as from terminal)
# start with empty list
file_names = []
# iterate through all file names found in that path
for file in os.listdir(path):
    # append all files names to a list
    if file[0]!='.' and file[-3:]=='pdf': #to handle issue with hidden files and non-pdf files
        file_names.append(file)
file_names

['N°\xa00007-unlocked.pdf',
 'N°\xa00006-unlocked.pdf',
 'N°\xa00009-unlocked.pdf',
 'N°\xa00005-unlocked.pdf',
 'N°\xa00004-unlocked.pdf']

In [23]:
# Step 2: (can be skipped if text files exists) Extract text from PDFs and save it to text
extracted_pdfs = []
                  
for file_name in file_names: 
    text = str(textract.process(path+file_name))
    # each extracted text is saved in .txt with same file name
    with open(path+file_name+'.txt','w+') as f:
        f.write(text)

In [65]:
# Step 3: Find patters in file, write search results to dictionary
search_terms = ['parts bénéficiaires','part bénéficiaires', 'parts beneficiaires', 
                    'part beneficiaires', 'beneficial units', 'beneficial unit', 'projet de fusion']

empty_results = {term:None for term in search_terms}
empty_all_results = {file_name:None for file_name in file_names}


for file_name in file_names:
    with open(path+file_name+'.txt','r') as f:
        text = f.read()
    all_results = empty_all_results
    results = empty_results.copy()
    for term in search_terms:
        search = re.findall(term, text)
        results[term]=len(search)
    all_results[file_name]=results
    pickle.dump(all_results,open(path+'all_search_results.pickle','wb'))
import pprint
pprint.pprint(all_results)

{'N°\xa00004-unlocked.pdf': {'alors': 1, 'amor': 3, 'amour': 0, 'les': 575},
 'N°\xa00005-unlocked.pdf': {'alors': 4, 'amor': 3, 'amour': 0, 'les': 571},
 'N°\xa00006-unlocked.pdf': {'alors': 0, 'amor': 4, 'amour': 0, 'les': 418},
 'N°\xa00007-unlocked.pdf': {'alors': 2, 'amor': 4, 'amour': 0, 'les': 550},
 'N°\xa00009-unlocked.pdf': {'alors': 0, 'amor': 4, 'amour': 0, 'les': 520}}


In [66]:
# Step 4: Copy files with more than 0 keywords to a new folder
from shutil import copyfile

all_results = pickle.load(open(path+'all_search_results.pickle','rb'))
for file in all_results:
    search_terms = list(all_results[file].keys())
    break
    
print('Will created folders for the following search terms: ', search_terms)

for term in search_terms:
    new_folder_name = term
    if os.path.exists(path+new_folder_name)==False:
        os.makedirs(path+new_folder_name)
    
    for file_name in all_results:
        match_quantity = all_results[file_name][term]
        if match_quantity>0:
            copyfile(path+file_name,path+new_folder_name+'/'+file_name)
            print('File ', file_name, ' moved to folder ', term)

Will created folders for the following search terms:  ['les', 'alors', 'amor', 'amour']
File  N° 0007-unlocked.pdf  moved to folder  les
File  N° 0006-unlocked.pdf  moved to folder  les
File  N° 0009-unlocked.pdf  moved to folder  les
File  N° 0005-unlocked.pdf  moved to folder  les
File  N° 0004-unlocked.pdf  moved to folder  les
File  N° 0007-unlocked.pdf  moved to folder  alors
File  N° 0005-unlocked.pdf  moved to folder  alors
File  N° 0004-unlocked.pdf  moved to folder  alors
File  N° 0007-unlocked.pdf  moved to folder  amor
File  N° 0006-unlocked.pdf  moved to folder  amor
File  N° 0009-unlocked.pdf  moved to folder  amor
File  N° 0005-unlocked.pdf  moved to folder  amor
File  N° 0004-unlocked.pdf  moved to folder  amor
